In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import rc, font_manager
ticks_font = font_manager.FontProperties(family='serif', style='normal',
                                         size=24, weight='normal', stretch='normal')

from pymatgen.core import Structure, Site,PeriodicSite,Lattice,Species,IStructure
import os
from pymatgen.io.cif import CifFile, CifParser, CifWriter
import mycif
from pymatgen.io.vasp import *
import crystal_toolkit
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import alphashape
from scipy.spatial import ConvexHull, Delaunay
from pymatgen.analysis.structure_matcher import StructureMatcher
import spglib


In [2]:
from copy import deepcopy

In [3]:
from pymatgen.analysis.molecule_structure_comparator import CovalentRadius

In [4]:
%load_ext autoreload
%autoreload 2
from kagome_tools import *
import ast
import plotly.graph_objects as go

In [5]:
#import warnings
#warnings.filterwarnings('ignore')
from itertools import permutations
import itertools

In [6]:
import random
import importlib
from tqdm.notebook import tqdm, trange
import time 
import ast
from pymatgen.analysis.chemenv.utils.coordination_geometry_utils import Plane

In [457]:
data_expand = pd.read_csv('./bulk_1_all_grouped_multi_gph_expand_2_20241111.csv')

In [458]:
mask_G2 = (data_expand['CN_group_index']==2)&(data_expand['angle_labels']=='Measured')
G_2_index = list(data_expand[mask_G2].index[:])

In [459]:
data_expand[mask_G2]

,Unnamed: 0,cifs_ids,cifs_name,space_group_num,num_atom,composition,unique_gph_files,plane_fun_gph,CN_plane,CN_group_index,...,angles_60_data,distance_60_data,distance_mean_120,distance_std_120,angles_mean_120,angles_std_120,angles_120_data,distance_120_data,unique_angles,net_names
0,0,intermetallics_search_000071.cif,intermetallics_search_000071.cif,166,30,"defaultdict(<class 'float'>, {'Fe0+': 18.0, 'S...",Fe3Sn2-71-Cagome_big.gph,"[-0.10606060825053208, -0.06123412072386011, 1...","[6, 6, 6, 6]",2.0,...,"[60.0, 60.0, 59.074, 59.074, 61.091, 61.091, 6...","[2.59024, 2.75376, 2.67911, 2.67911, 2.67909, ...",4.63033,0.03339,119.83550,0.73868,"[120.0, 120.0, 120.926, 120.926, 118.909, 118....","[4.62876, 4.62876, 4.58473, 4.58473, 4.67907, ...","[57.817, 59.074, 60.0, 61.091, 61.852, 118.909...",Likely_Filled
18,18,intermetallics_search_002414.cif,intermetallics_search_002414.cif,166,36,"defaultdict(<class 'float'>, {'Hf0+': 9.0, 'Ni...",HfNi3-2414-Cagome_big.gph,"[9.235182172113557e-17, 1.599580473925498e-16,...","[6, 6, 6, 6]",2.0,...,"[60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60....","[2.63935, 2.63935, 2.63935, 2.63935, 2.63935, ...",4.57149,0.00000,120.00000,0.00000,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120...","[4.57149, 4.57149, 4.57149, 4.57149, 4.57149, ...","[60.0, 120.0]",Likely_Filled
19,19,intermetallics_search_002414.cif,intermetallics_search_002414.cif,166,36,"defaultdict(<class 'float'>, {'Hf0+': 9.0, 'Ni...",HfNi3-2414-Cagome_big_1.gph,"[0.0, -5.462017949042774e-16, 2.62015166647399...","[6, 6, 6, 6]",2.0,...,"[64.45, 64.45, 64.45, 55.465, 55.465, 55.465, ...","[2.81883, 2.81883, 2.81883, 2.45989, 2.45989, ...",4.57374,0.07326,119.95725,1.58930,"[119.872, 119.872, 119.872, 119.872, 119.872, ...","[4.57501, 4.57501, 4.57501, 4.57501, 4.57501, ...","[55.465, 57.775, 60.0, 62.268, 64.45, 117.732,...",Likely_Filled
20,20,intermetallics_search_002415.cif,intermetallics_search_002415.cif,194,40,"defaultdict(<class 'float'>, {'Hf0+': 10.0, 'N...",HfNi3-2415-Cagome_big.gph,"[9.44441886217379e-16, 5.452737772415611e-16, ...","[6, 6, 6, 6]",2.0,...,"[64.642, 64.642, 64.642, 55.358, 55.358, 55.35...","[2.82651, 2.82651, 2.82651, 2.45569, 2.45569, ...",4.57640,0.07572,120.00000,1.64119,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120...","[4.57828, 4.57828, 4.57828, 4.57828, 4.57828, ...","[55.358, 57.679, 60.0, 62.321, 64.642, 117.679...",Likely_Filled
21,21,intermetallics_search_002415.cif,intermetallics_search_002415.cif,194,40,"defaultdict(<class 'float'>, {'Hf0+': 10.0, 'N...",HfNi3-2415-Cagome_big_2.gph,"[1.3509439757680297e-16, 7.799678680697746e-17...","[6, 6, 6, 6]",2.0,...,"[64.159, 64.159, 64.159, 55.706, 55.706, 55.70...","[2.81015, 2.81015, 2.81015, 2.47209, 2.47209, ...",4.57687,0.06899,119.93225,1.49622,"[119.797, 119.797, 119.797, 119.797, 119.797, ...","[4.57765, 4.57765, 4.57765, 4.57765, 4.57765, ...","[55.706, 57.921, 60.0, 62.147, 64.159, 117.853...",Likely_Filled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17754,17754,intermetallics_search_687392.cif,intermetallics_search_687392.cif,225,16,"defaultdict(<class 'float'>, {'Fe0+': 12.5, 'A...",Fe3.125Al0.875-687392-Cagome_big_1.gph,"[-0.22880277777777774, -0.2288027777777778, 0....","[6, 6, 6, 6, 6, 6]",2.0,...,"[54.736, 54.736, 54.736, 54.736, 70.529, 70.52...","[2.48551, 2.48551, 2.48551, 2.48551, 2.87, 2.8...",4.52583,0.33025,119.99967,7.44489,"[125.264, 125.264, 125.264, 125.264, 109.471, ...","[4.75935, 4.75935, 4.75935, 4.75935, 4.05878, ...","[54.736, 70.529, 109.471, 125.264]",Likely_Filled
17755,17755,intermetallics_search_687392.cif,intermetallics_search_687392.cif,225,16,"defaultdict(<class 'float'>, {'Fe0+': 12.5, 'A...",Fe3.125Al0.875-687392-Cagome_big_2.gph,"[-1.874317694514235e-17, 0.24226176470588232, ...","[6, 6, 6, 6, 6, 6]",2.0,...,"[54.736, 54.736, 54.736, 54.736, 70.529, 70.52...","[2.48551, 2.48551, 2.48551, 2.48551, 2.87, 2.8...",4.52583,0.33025,119.99967,7.44489,"[125.264, 125.264,

In [460]:
def transmat(lattice_A,lattice_B):
    # transformation from lattice A to lattice B
    M_A_B = np.dot(lattice_B, np.linalg.inv(lattice_A))
    return M_A_B
def clean_file(gphfile):
    with open(gphfile) as fh:
        lines = fh.readlines()
        
    vec_info = np.array([np.float64(x) for x in lines[1].strip('\n').split(' ')])
    #print(vec_info)
    site_list = []
    lattice = Lattice.from_parameters(a=vec_info[0],b=vec_info[1],c=vec_info[2],alpha=vec_info[3],beta=vec_info[4],gamma=vec_info[5])
    for line in lines[2:]:
        
        ls = line.strip('\n').split(' ')
        
        coor_txt = [ls[i+2] for i in range(3)]
        ele = Species(ls[1])
        coor = np.array([np.float64(x) for x in coor_txt])
        
        site = PeriodicSite(ele,coords=coor,lattice=lattice)
        site_list.append(site)
    structure = Structure.from_sites(site_list)
    return structure
def get_site_idx(structure_sub,structure_full):
    site_sub = structure_sub.sites
    site_full = structure_full.sites
    
    sub_full_ids = []
    for i in range(len(site_sub)):
        site_i = site_sub[i]
        for j in range(len(site_full)):
            site_j = site_full[j]
            dist = np.linalg.norm(site_i.coords-site_j.coords)
            if np.round(dist,1)==0:
                #print(dist, site_i,site_j)
                sub_full_ids.append(j)
                
    return sub_full_ids

In [461]:
# start looping to get the structures stored in a list: 

structure_full_list = []
structure_sub_list = []
structure_full_n_atom_list = []
gph_top = '../Intermetallics_kagome_search_graph/'
top_dir = '../../intermetallics_search_results2/'

p_bar = tqdm(data_expand[mask_G2].index)
#for i in range(len(sorted_cifs_ids_short)):
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    cif_file = data_expand['cifs_name'][i]
    try:
        cif = CifParser(top_dir+cif_file,occupancy_tolerance=100,site_tolerance=0.008)
        structure_full_orig = cif.get_structures(primitive=False)[0]
    except:
        cif = CifParser(top_dir+cif_file,occupancy_tolerance=101,site_tolerance=0.008)
        structure_full_orig = cif.get_structures(primitive=False)[0]
    
    structure_full = structure_full_orig
    structure_sub_ = []
    
    gph_fn = data_expand['unique_gph_files'][i]
    structure_sub = clean_file(gph_top+gph_fn)
    #
    #new_structure_sub = Structure.from_sites(structure_sub.sites)
    
            
    
    structure_full_list.append(structure_full)
    structure_sub_list.append(structure_sub)
    
    

  0%|          | 0/2575 [00:00<?, ?it/s]

In [464]:
all_jimages = get_all_images()

In [463]:
#filtered_3_remain_index = list(filtered_3_data.index)

# filter round 4
# build a planar structure based on the sites. 
filter_4_match_res = [] # document the ABCD of that plane
plane_structures_list = []
plane_structures_pbc_list = []
p_bar = tqdm(data_expand[mask_G2].index)
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    st_i = G_2_index.index(i)
    structure_full = structure_full_list[st_i]        
    pop_sites = populate_sites(structure_full,all_jimages)

    
    
    structure_sub = structure_sub_list[st_i]
    #A,B,C,D = get_plane_from_sites_m3(structure_sub.sites)
    A,B,C,D = ast.literal_eval(data_expand['plane_fun_gph'][i])
    
    # get_plane_from_sites_m is not good enough
    plane_my = Plane([A,B,C,-D])
    #for site in structure_sub.sites:
    #    print(calc_site_plane_distance(A,B,C,D,site))
    sites_in_plane,sites_plane_dist = get_sites_in_plane(structure_full,pop_sites,A,B,C,D)
    plane_sites = [structure_full.sites[x] for x in sites_in_plane]
    sites_in_gph, sites_out_gph = get_site_in_out_gph(plane_sites,structure_sub)
    # group atoms in the supercell based on the distance
    points = [s.coords for s in plane_sites]
    plane_dist_site_group = plane_my.distances_indices_groups(points,0.15)
    # this gives us the sites with approximately the same distances to the plane
    # so that I know these sites should be on the gph, but gph may exclude those sites


    #print(np.round(plane_dist_site_group[0],2),plane_dist_site_group[2])
    if len(plane_dist_site_group[2])>1:
        num_sites_group = [(len(x)) for x in plane_dist_site_group[2]]
        more_sites_group = num_sites_group.index(max(num_sites_group))
        plane_sites_clean = [plane_sites[int(i)] for i in plane_dist_site_group[2][more_sites_group]]
    else:
        plane_sites_clean = plane_sites
    # get plane_sites
    # get A,B,C,D
    if len(plane_sites_clean)<3:
        #number of atoms <3
        nA,nB,nC,nD,nplane_sites = [A,B,C,D,plane_sites]
    elif len(plane_sites_clean)==3:
        if points_collinear(plane_sites_clean):
            nA,nB,nC,nD,nplane_sites = [A,B,C,D,plane_sites]
        else:
            # number of atoms ==3
            nA,nB,nC,nD,nplane_sites = get_plane_from_sites(plane_sites_clean)
    else:
        # number of atoms > 3
        nA,nB,nC,nD = get_plane_from_sites_m3(plane_sites_clean)
    #match_fragment_plane.append([nA,nB,nC,nD]) 
    # make a cleaner plane_structure, this is crucial to further steps. 
    # 1. check if other sites are missed on the facets/edges 
    plane_sites_clean_ = [s for s in plane_sites_clean]
    if len(plane_sites_clean)<len(plane_sites):
        for site in plane_sites:
            if site not in plane_sites_clean_:
                if 0 in [np.round(x,1) for x in site.frac_coords]:
                    plane_sites_clean_.append(site)
                elif 1 in [np.round(x,1) for x in site.frac_coords]:
                    plane_sites_clean_.append(site)
    # 2. add atoms if needed  
    plane_func = Plane([nA,nB,nC,-nD])
    if len(plane_sites_clean)<len(plane_sites_clean_):
        plane_sites_cleaned,plane_sites_pbc = get_plane_sites_clean(plane_sites,plane_sites_clean,all_jimages,plane_func)
        if len(plane_sites_cleaned)>3:
            nA,nB,nC,nD = get_plane_from_sites_m3(plane_sites_cleaned)
    else:
        #plane_sites_cleaned = get_plane_sites_clean_m2(plane_sites_clean)
        plane_sites_cleaned,plane_sites_pbc = get_plane_sites_clean(plane_sites,plane_sites_clean,all_jimages,plane_func)
        if len(plane_sites_cleaned)>3:
            nA,nB,nC,nD = get_plane_from_sites_m3(plane_sites_cleaned)
    # get plane structure
    plane_structure = Structure.from_sites(plane_sites_cleaned)

    # get plane structure with PBC 
    plane_structure_pbc = Structure.from_sites(plane_sites_pbc)



    filter_4_match_res.append([nA,nB,nC,nD])
    plane_structures_list.append(plane_structure)
    plane_structures_pbc_list.append(plane_structure_pbc)
    

  0%|          | 0/2575 [00:00<?, ?it/s]

In [15]:
'''
plane_structures_list = []
plane_structures_pbc_list = []
pbc_folder = './plane_structures_pbc_gph/'
pls_folder = './plane_structures_gph/'
G_2_index = list(data_expand[mask_G2].index[:])
pbar = tqdm(G_2_index)
for i in pbar:
    filename = data_expand['unique_gph_files'][i].split('.gph')[0]
    read_pbc = pbc_folder+filename+'.cif'
    read_pls = pls_folder+filename+'.cif'
    try:
        pbc_cif = mycif.CifParser(read_pbc,occupancy_tolerance=100,site_tolerance=0.0001,round_frac=False)
        plane_structure_pbc = pbc_cif.get_structures(primitive=False)[0]
    except:
        pbc_cif = mycif.CifParser(read_pbc,occupancy_tolerance=100,site_tolerance=0.0001,round_frac=False)
        plane_structure_pbc = pbc_cif.get_structures(primitive=False)[0]
    plane_structures_pbc_list.append(plane_structure_pbc)
    
    try:
        pls_cif = mycif.CifParser(read_pls,occupancy_tolerance=100,site_tolerance=0.0001,round_frac=False)
        plane_structure = pls_cif.get_structures(primitive=False)[0]
    except:
        pls_cif = mycif.CifParser(read_pls,occupancy_tolerance=100,site_tolerance=0.0001,round_frac=False)
        plane_structure = pls_cif.get_structures(primitive=False)[0]
    plane_structures_list.append(plane_structure)


  0%|          | 0/2596 [00:00<?, ?it/s]

In [468]:
def check_plane_structure(structure):
    # check the composition of a structure 
    # need to tell whether all the sites in this structure are compositionally identical
    composition = structure.composition.as_dict()
    if len(structure.sites)==1 or len(composition.keys())==1:
        # hexagonal because only one site in the plane or the composition consists only one element
        # done
        return 'Likely Hexagonal'    
    
    else:
        # the rest of the structures can be:
        # 1. filled kagome - composition is A3-B
        # 2. filled kagome - composition is (A,B)3-(C,D)
        # 3. other types of sublattices: 
        #    hexagonal + honeycomb
        #    three or more sublattice sites. 
        # 
        # check sublattice sites based on the composition of each site
        site_composition = [site.species.as_dict() for site in structure]
        unique_site_comp = [] # get unique compositions from all the sites 
        for s_comp in site_composition:
            if s_comp not in unique_site_comp:
                unique_site_comp.append(s_comp) 
        unique_site_comp_count = [] # count the number of sites with the same composition 
        unique_site_comp_group = [] # group the index of the sites with the same composition
        
        for uni_s_comp in unique_site_comp:
            count = 0
            site_comp_group = []
            for i,s_comp in enumerate(site_composition):
                if s_comp == uni_s_comp:
                    count+=1
                    site_comp_group.append(i)
            unique_site_comp_group.append(site_comp_group)
            unique_site_comp_count.append(count) 
        #print(unique_site_comp,len(unique_site_comp),unique_site_comp_count,unique_site_comp_group)
        
        if len(unique_site_comp) == 2:
            # two sublattices 
            if unique_site_comp_count[0]/unique_site_comp_count[1] in [1/3,3]:
                return 'Likely filled - 2'
            else: 
                return 'Likely others - 2'
        if len(unique_site_comp) == 3:
            return 'Likely others - 3'
        
        if len(unique_site_comp) == 4:
            return 'Likely others - 4'
        
        else: 
            return 'Likely others - x'
        
        

In [469]:
def make_empty_net(structure):
    # for a potentially filled kagome net, take out the atoms and make an empty net structure. 
    composition = structure.composition.as_dict()
    site_composition = [site.species.as_dict() for site in structure]
    unique_site_comp = [] # get unique compositions from all the sites 
    for s_comp in site_composition:
        if s_comp not in unique_site_comp:
            unique_site_comp.append(s_comp) 
    unique_site_comp_count = [] # count the number of sites with the same composition 
    unique_site_comp_group = [] # group the index of the sites with the same composition

    for uni_s_comp in unique_site_comp:
        count = 0
        site_comp_group = []
        for i,s_comp in enumerate(site_composition):
            if s_comp == uni_s_comp:
                count+=1
                site_comp_group.append(i)
        unique_site_comp_group.append(site_comp_group)
        unique_site_comp_count.append(count) 
    #print(unique_site_comp,len(unique_site_comp),unique_site_comp_count,unique_site_comp_group)
    
    if unique_site_comp_count[0]>unique_site_comp_count[1]:
        sites_net_index = 0
    else: 
        sites_net_index = 1
    
    new_sites = [structure.sites[i] for i in unique_site_comp_group[sites_net_index]]
    
    new_structure = Structure.from_sites(new_sites)
    return new_structure

In [470]:
#oth_index_2[91]

In [471]:
# analyze whether the plane structure is a hexagonal close-packed plane 
test_labels_pbc = []
test_labels_pls = []
pbar = tqdm(G_2_index)
for i in pbar:
    st_i = G_2_index.index(i)
    natom = data_expand['num_atom'][i]
    
    plane_structure_pbc = plane_structures_pbc_list[st_i]
    
    label1 = check_plane_structure(plane_structure_pbc)
    
    plane_structure = plane_structures_list[st_i]
    label2 = check_plane_structure(plane_structure)
    
    #if label1!=label2:
    #    print(label1,label2,natom,i,data_expand['cifs_name'][i],data_expand['space_group_num'][i])
    if natom<6:
        if 'Likely filled - 2' in [label1,label2] :
            # these are not likely filled
            print(label1,label2,natom,i,data_expand['cifs_name'][i],data_expand['space_group_num'][i])
            label1 = 'Likely cubic'
            label2 = 'Likely cubic'
    test_labels_pbc.append(label1)        
    test_labels_pls.append(label2)

  0%|          | 0/2575 [00:00<?, ?it/s]

Likely others - 2 Likely filled - 2 5 7396 intermetallics_search_194417.cif 221
Likely others - 2 Likely filled - 2 5 7398 intermetallics_search_194420.cif 221
Likely others - 2 Likely filled - 2 5 7400 intermetallics_search_194423.cif 221
Likely others - 2 Likely filled - 2 5 7426 intermetallics_search_194968.cif 99
Likely others - 2 Likely filled - 2 5 7428 intermetallics_search_194968.cif 99
Likely others - 2 Likely filled - 2 5 7430 intermetallics_search_194970.cif 99
Likely others - 2 Likely filled - 2 5 7431 intermetallics_search_194970.cif 99
Likely others - 2 Likely filled - 2 5 7432 intermetallics_search_194970.cif 99
Likely others - 2 Likely filled - 2 5 7433 intermetallics_search_194971.cif 99
Likely others - 2 Likely filled - 2 5 7434 intermetallics_search_194971.cif 99
Likely others - 2 Likely filled - 2 5 7435 intermetallics_search_194971.cif 99
Likely others - 2 Likely filled - 2 5 16929 intermetallics_search_680381.cif 221
Likely others - 2 Likely filled - 2 5 16930 int

In [472]:
#data_expand.iloc[G_2_index][data_expand.iloc[G_2_index]['num_atom']<6]

In [473]:
hex_index = [i for i in range(len(test_labels_pls)) if 'Hex' in test_labels_pbc[i]]

In [474]:
fil_index = [i for i in range(len(test_labels_pls)) if 'Likely filled - 2' in [test_labels_pbc[i],test_labels_pls[i]]]

In [475]:
#fil_index.index(851)

In [476]:
oth_index_2 = [i for i in range(len(test_labels_pls)) if 'Likely others - 2' in [test_labels_pbc[i], test_labels_pls[i]]]
oth_index_3 = [i for i in range(len(test_labels_pls)) if 'Likely others - 3' in [test_labels_pbc[i], test_labels_pls[i]]]
oth_index_4 = [i for i in range(len(test_labels_pls)) if 'Likely others - 4' in [test_labels_pbc[i], test_labels_pls[i]]]
oth_index_x = [i for i in range(len(test_labels_pls)) if ('Likely others - x'==test_labels_pbc[i]) and ('Likely others - x'==test_labels_pls[i])]

oth_index = [i for i in range(len(test_labels_pls)) if ('Likely others' in test_labels_pbc[i]) or ('Likely others' in test_labels_pls[i])]


In [481]:
def get_1nn_m2(structure,
            plane_structure_very_clean,
            plane_func_coeffs,
            NN_method=CrystalNN(x_diff_weight=0.0, porous_adjustment=False )):
    # get the 1NN for each site within the radius r_d
    A,B,C,D = plane_func_coeffs
    plane_handle = Plane([A,B,C,-D])
    sites = plane_structure_very_clean.sites
    sites_nn_info = {}
    B_NN_info_pre = NN_method.get_all_nn_info(structure=structure) 
    B_NN_info = [[] for x in sites]
    
    for i in range(len(sites)):
        site_i_it = structure.sites.index(sites[i])
        nn_site_index = range(len(B_NN_info_pre[site_i_it]))
        #print(site_i_it)
        for j in nn_site_index:
            site_nn_j = B_NN_info_pre[site_i_it][j]
            plane_to_point = np.abs(plane_handle.distances_indices_groups([site_nn_j['site'].coords])[0][0])
            #if i!=site_nn_j['site_index']:
            if plane_to_point<0.4:
                #print(i)
                B_NN_info[i].append(site_nn_j)

    
    #sites_coordination = [len(x) for x in B_NN_info]
    #return sites,B_NN_info
    
    for i in range(len(sites)):
        nn_site_index = range(len(B_NN_info[i]))   #np.unique([x['site_index'] for x in B_NN_info[i]])
        angles_index_list = [(x[0],i,x[1]) for x in list(itertools.combinations(nn_site_index,2))]
        #print(angles_index_list)
        for j in range(len(B_NN_info[i])):
            a = sites[i].coords 
            b = B_NN_info[i][j]['site'].coords
            B_NN_info[i][j]["distance"] = np.round(np.linalg.norm(a-b),5) #np.round(sites[i].distance(B_NN_info[i][j]['site']),5)
        #print(len(B_NN_info[i]))
        angle_info = []
        for x in angles_index_list:
            if calc_angle(B_NN_info[i][x[0]]['site'],sites[i],B_NN_info[i][x[2]]['site']) not in [0,180]:
                # (j,i,k)
                j = B_NN_info[i][x[0]]['site_index']
                k = B_NN_info[i][x[2]]['site_index']
                # angle(j,i,k)
                angle = calc_angle_m(B_NN_info[i][x[0]]['site'],sites[i],B_NN_info[i][x[2]]['site'])
                # distances
                dij = B_NN_info[i][x[0]]['distance']
                dik = B_NN_info[i][x[2]]['distance']
                djk = np.round(calculate_opposite_side_length(angle,dij,dik),5)
                angle_info.append([j,i,k, angle, dij,dik,djk] )
                
        angle_info = [x for x in angle_info if str(x[3])!='nan' ]
        sites_nn_info['{}'.format(str(i))]={"nn_info":B_NN_info[i],
                                            "angle_info":angle_info}
    return sites, sites_nn_info

In [482]:
G_2_index.index(5097)

516

In [486]:
# finer filter on filled nets 
oth_index_2_survive = []
pbar = tqdm(G_2_index)
for i in pbar:
    st_i = G_2_index.index(i)
    if st_i in oth_index_2:
        # potentially filled index, 
        # then we measure it carefully. 
        # 
        nn_sites = None
        nn_info = None
        plane_structure_pbc = plane_structures_pbc_list[st_i]
        
        plane_structure = plane_structures_list[st_i]
        
        new_plane_structure_pbc = make_empty_net(plane_structure_pbc)
        new_plane_structure = make_empty_net(plane_structure)
        #absds = ast.literal_eval(data_expand['plane_fun_gph'][i])
        absds = filter_4_match_res[st_i]
        [A,B,C,D] = absds
        for cutoff in range(7,30):
            try:
                # Try to call your function with the current parameter
                C_NN = CrystalNN(distance_cutoffs=(2,2.4),x_diff_weight=0.0, porous_adjustment=False,search_cutoff=cutoff )
                nn_sites,nn_info = get_1nn_m2(new_plane_structure_pbc,
                                   new_plane_structure,
                                   [A,B,C,D],
                                   NN_method=C_NN
                                  )
                #cutoff_.append(cutoff)
                break
            except Exception as e:
                # If there is an error, print it and continue to the next parameter
                continue
                
        try:
            keys = nn_info.keys()
            CN = [len(nn_info[k]['nn_info']) for k in keys]
            unique_CN = np.unique(CN)
            if len(unique_CN)==1 and 4 in unique_CN:
                oth_index_2_survive.append(st_i)
        except:
            continue


  0%|          | 0/2575 [00:00<?, ?it/s]

In [496]:
G_2_angles = []
G_2_angle_distance_info_all = []
G_2_CN = []
G_2_CN_strange_index = []
G_2_CN_strange_index_it = []
G_2_CN_strange = []
pbar = tqdm(G_2_index)

for i in pbar:
    st_i = G_2_index.index(i)
    if st_i in fil_index+oth_index_2_survive:
        # potentially filled index, 
        # then we measure it carefully. 
        # 
        nn_sites = None
        nn_info = None
        plane_structure_pbc = plane_structures_pbc_list[st_i]
        
        plane_structure = plane_structures_list[st_i]
        
        new_plane_structure_pbc = make_empty_net(plane_structure_pbc)
        new_plane_structure = make_empty_net(plane_structure)
        #absds = ast.literal_eval(data_expand['plane_fun_gph'][i])
        absds = filter_4_match_res[st_i]
        [A,B,C,D] = absds
        for cutoff in range(7,30):
            try:
                # Try to call your function with the current parameter
                C_NN = CrystalNN(distance_cutoffs=(2,2.4),x_diff_weight=0.0, porous_adjustment=False,search_cutoff=cutoff )
                nn_sites,nn_info = get_1nn_m2(new_plane_structure_pbc,
                                   new_plane_structure,
                                   [A,B,C,D],
                                   NN_method=C_NN
                                  )
                #cutoff_.append(cutoff)
                break
            except Exception as e:
                # If there is an error, print it and continue to the next parameter
                continue
        keys = nn_info.keys()
        G_2_angles.append(np.unique(np.concatenate([[x[3] for x in nn_info[k]['angle_info']]  for k in keys])))
        G_2_angle_distance_info_all.append([ nn_info[k]['angle_info'] for k in keys])
        CN = [len(nn_info[k]['nn_info']) for k in keys]
        G_2_CN.append(CN)
        if len(np.unique(CN)) != 1 or 0 in CN or 4 not in CN:
            G_2_CN_strange_index.append(i)
            G_2_CN_strange_index_it.append(st_i)
            G_2_CN_strange.append(CN)
            
    else:
        G_2_angles.append('NA')
        G_2_angle_distance_info_all.append('NA')
        G_2_CN.append('NA')

  0%|          | 0/2575 [00:00<?, ?it/s]

In [497]:
new_fil_index = [x for x in fil_index+oth_index_2_survive if x not in G_2_CN_strange_index_it]

In [501]:
#data_expand.iloc[G_2_CN_strange_index]

In [502]:
# define functions to analyze std of angles around an atom and atomic distances
def quantity_std(site_info):
    '''
    site_info: a list of sites information containing atom index, angles, and distances
    site_info[i][ii]=[j,i,k,angle,dij,dik,djk] 
    return: site grouped angles and distances
    '''
    l_i = range(len(site_info))
    all_si_di = []
    all_si_angles = []
    for i in l_i:
        si = np.array(site_info[i])
        si_angles = si.T[3]
        jk = [] # site-j and site-k index
        si_di = [] # distance around site-i
        for j in range(len(si.T[0])):
            j_a = si.T[0][j]
            if j_a not in jk:
                jk.append(j_a)
                si_di.append(si.T[4][j])
        for k in range(len(si.T[2])):
            k_a = si.T[2][k]
            if k_a not in jk:
                jk.append(k_a)
                si_di.append(si.T[4][k])
        #print(jk)
        all_si_di.append(np.array(si_di))
        all_si_angles.append(si_angles)
    all_si_di = np.array(all_si_di)
    all_si_angles = np.array(all_si_angles)
    return all_si_di,all_si_angles

# define functions to get the angles and distances around specified sites-i
def get_angles_distances_jik(site_info):
    '''
    site_info: a list of sites information containing atom index, angles, and distances
    site_info[i][ii]=[j,i,k,angle,dij,dik,djk] 
    return: site grouped Angles_jik and Distances-djk
    '''
    l_i = range(len(site_info))
    all_si_di = []
    all_si_angles = []
    for i in l_i:
        si = np.array(site_info[i])
        si_angles = si.T[3]
        si_djk = si.T[-1]
        
        all_si_di.append(np.array(si_djk))
        all_si_angles.append(si_angles)
    all_si_di = np.array(all_si_di)
    all_si_angles = np.array(all_si_angles)
    return all_si_di,all_si_angles
# function to group the nearest sites according to angles from get_angles_distances_jik
def get_quantity_std(all_site_distances,all_site_angles,angles_around=60,delta=40):
    '''
    all_site_distances,all_site_angles: np.arrays from get_angles_distances_jik
    angels_around: only return distances when the angles are roughtly equal to this value.
    delta: difference between max and min angles for grouping the angles. 
    '''
    l_i = range(len(all_site_distances))
    selected_angles = np.array([])
    selected_djk = np.array([])
    for i in l_i:
        #print(selected_angles)
        si_angles = all_site_angles[i]#-angles_around
        si_angles_abs = np.abs(all_site_angles[i]-angles_around)
        si_djk = all_site_distances[i]
        
        angles_abs_sort,angles_abs_group,angles_abs_group_index = group_angles(si_angles_abs,delta=delta)
        #print(np.array([si_angles[int(index)] for index in angles_abs_group_index[0]]))
        selected_angles = np.concatenate([selected_angles,np.array([si_angles[int(index)] for index in angles_abs_group_index[0]])])
        selected_djk = np.concatenate([selected_djk,np.array([si_djk[int(index)] for index in angles_abs_group_index[0]])])
    return selected_angles,selected_djk

In [503]:
data_expand_G2 = deepcopy(data_expand[mask_G2])

In [504]:
G2_data_angles_around_60_mean = []
G2_data_angles_around_60_std = []
G2_data_djk_around_60_mean = []
G2_data_djk_around_60_std = []
G2_data_angles_around_60 = []
G2_data_djk_around_60 = []

# no data: 'NA'
for i in data_expand_G2.index:
    iG = G_2_index.index(i)
    #print(i)
    if iG in new_fil_index:
        #print(iG)
        
        
        G_2_angle_distance_info_ = G_2_angle_distance_info_all[iG]
        #########
        test_di,test_angles = get_angles_distances_jik(G_2_angle_distance_info_)
        G2_angles_jik, G2_djk = get_quantity_std(test_di,test_angles,angles_around=60,delta=20)

        G2_data_djk_around_60_std.append(np.round(np.std(G2_djk),4))
        G2_data_angles_around_60_std.append(np.round(np.std(G2_angles_jik),4))
        G2_data_djk_around_60_mean.append(np.round(np.mean(G2_djk),4))
        G2_data_angles_around_60_mean.append(np.round(np.mean(G2_angles_jik),4))
        G2_data_angles_around_60.append(list(np.round(G2_angles_jik,4)))
        G2_data_djk_around_60.append(list(np.round(G2_djk,4)))
        ##########
        
    else:
        G2_data_djk_around_60_std.append('N/A')
        G2_data_angles_around_60_std.append('N/A')
        G2_data_djk_around_60_mean.append('N/A')
        G2_data_angles_around_60_mean.append('N/A')
        G2_data_angles_around_60.append('N/A')
        G2_data_djk_around_60.append('N/A')

In [505]:
i,iG

(17776, 2574)

In [506]:
G_2_angle_distance_info_

[[[1, 0, 2, 120.0, 2.8425, 2.8425, 4.92335],
  [1, 0, 2, 60.0, 2.8425, 2.8425, 2.8425],
  [2, 0, 1, 60.0, 2.8425, 2.8425, 2.8425],
  [2, 0, 1, 120.0, 2.8425, 2.8425, 4.92335]],
 [[0, 1, 2, 120.0, 2.8425, 2.8425, 4.92335],
  [0, 1, 2, 60.0, 2.8425, 2.8425, 2.8425],
  [2, 1, 0, 60.0, 2.8425, 2.8425, 2.8425],
  [0, 1, 2, 120.0, 2.8425, 2.8425, 4.92335]],
 [[0, 2, 1, 60.0, 2.8425, 2.8425, 2.8425],
  [0, 2, 1, 120.0, 2.8425, 2.8425, 4.92335],
  [0, 2, 1, 120.0, 2.8425, 2.8425, 4.92335],
  [0, 2, 1, 60.0, 2.8425, 2.8425, 2.8425]]]

In [507]:
G2_data_angles_around_120_mean = []
G2_data_angles_around_120_std = []
G2_data_djk_around_120_mean = []
G2_data_djk_around_120_std = []
G2_data_angles_around_120 = []
G2_data_djk_around_120 = []

# no data: 'NA'
for i in data_expand_G2.index:
    iG = G_2_index.index(i)
    #print(i)
    if iG in new_fil_index:
        #print(iG)
        
        
        G_2_angle_distance_info_ = G_2_angle_distance_info_all[iG]
        #########
        test_di,test_angles = get_angles_distances_jik(G_2_angle_distance_info_)
        G2_angles_jik, G2_djk = get_quantity_std(test_di,test_angles,angles_around=120,delta=20)

        G2_data_djk_around_120_std.append(np.round(np.std(G2_djk),4))
        G2_data_angles_around_120_std.append(np.round(np.std(G2_angles_jik),4))
        G2_data_djk_around_120_mean.append(np.round(np.mean(G2_djk),4))
        G2_data_angles_around_120_mean.append(np.round(np.mean(G2_angles_jik),4))
        G2_data_angles_around_120.append(list(np.round(G2_angles_jik,4)))
        G2_data_djk_around_120.append(list(np.round(G2_djk,4)))
        ##########
        
    else:
        G2_data_djk_around_120_std.append('N/A')
        G2_data_angles_around_120_std.append('N/A')
        G2_data_djk_around_120_mean.append('N/A')
        G2_data_angles_around_120_mean.append('N/A')
        G2_data_angles_around_120.append('N/A')
        G2_data_djk_around_120.append('N/A')

In [508]:
for i in data_expand_G2.index[:]:
    iG = G_2_index.index(i)
    #print(i)
    if iG in new_fil_index:
        data_expand_G2['distance_mean_120'][i] = G2_data_djk_around_120_mean[iG]
        data_expand_G2['distance_std_120'][i] = G2_data_djk_around_120_std[iG]
        data_expand_G2['angles_mean_120'][i] = G2_data_angles_around_120_mean[iG]
        data_expand_G2['angles_std_120'][i] = G2_data_angles_around_120_std[iG]
        data_expand_G2['angles_120_data'][i] = G2_data_angles_around_120[iG]
        data_expand_G2['distance_120_data'][i] = G2_data_djk_around_120[iG]
        
        
        data_expand_G2['distance_mean_60'][i] = G2_data_djk_around_60_mean[iG]
        data_expand_G2['distance_std_60'][i] = G2_data_djk_around_60_std[iG]
        data_expand_G2['angles_mean_60'][i] = G2_data_angles_around_60_mean[iG]
        data_expand_G2['angles_std_60'][i] = G2_data_angles_around_60_std[iG]
        data_expand_G2['angles_60_data'][i] = G2_data_angles_around_60[iG]
        data_expand_G2['distance_60_data'][i] = G2_data_djk_around_60[iG]
        data_expand_G2['unique_angles'][i] = list(np.unique(G_2_angles[iG]))
        #print(data_expand_G2['angles_std_60'][i],G2_data_angles_around_60_std[iG])
        
        data_expand_G2['net_names'][i] = 'Likely Filled'
    else: 
        data_expand_G2['distance_mean_120'][i] = 'NA'
        data_expand_G2['distance_std_120'][i] = 'NA'
        data_expand_G2['angles_mean_120'][i] = 'NA'
        data_expand_G2['angles_std_120'][i] = 'NA'
        data_expand_G2['angles_120_data'][i] = 'NA'
        data_expand_G2['distance_120_data'][i] = 'NA'
        
        data_expand_G2['distance_mean_60'][i] = 'NA'
        data_expand_G2['distance_std_60'][i] = 'NA'
        data_expand_G2['angles_mean_60'][i] = 'NA'
        data_expand_G2['angles_std_60'][i] = 'NA'
        data_expand_G2['angles_60_data'][i] = 'NA'
        data_expand_G2['distance_60_data'][i] = 'NA'
        
        data_expand_G2['unique_angles'][i] = 'NA'
        
        if iG in hex_index:
            data_expand_G2['net_names'][i] = 'Likely Hexagonal'
        else:
            data_expand_G2['net_names'][i] = 'Others'

In [509]:
data_expand_G2.to_csv('./bulk_1_all_grouped_multi_gph_expand_2_filled_20241111.csv')